In [ ]:
%load_ext autoreload
%autoreload 2

import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import processor, hist

from processor.dielectron_mass_MC import dielectron_mass
from Tools.config_helpers import loadConfig
from klepto.archives import dir_archive

In [ ]:
from processor.default_accumulators import desired_output, add_processes_to_output

from Tools.helpers import get_samples
from Tools.config_helpers import redirector_ucsd, redirector_fnal
from Tools.nano_mapping import make_fileset, nano_mapping

overwrite = True
local = True

# load the config and the cache
cfg = loadConfig()

cacheName = 'dielectron_mass'
cache = dir_archive(os.path.join(os.path.expandvars(cfg['caches']['base']), cacheName), serialized=True)

year = 2016

# get a python dictionary of all NanoAOD samples
# The samples definitions can be found in data/samples.yaml
samples = get_samples(year)

# make a fileset, taking the definitions in Tools/nano_mapping.py
fileset = make_fileset(['DY', 'TT/TW', 'WZ/ZZ',], year, redirector=redirector_ucsd, small=False)

# in order for cutflows to work we need to add every process to the output accumulator
add_processes_to_output(fileset, desired_output)

histograms = sorted(list(desired_output.keys()))

if local:

    exe_args = {
        'workers': 16,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
    }
    exe = processor.futures_executor

else:
    from Tools.helpers import get_scheduler_address
    from dask.distributed import Client, progress

    scheduler_address = get_scheduler_address()
    c = Client(scheduler_address)

    exe_args = {
        'client': c,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
    }
    exe = processor.dask_executor


if not overwrite:
    cache.load()

if cfg == cache.get('cfg') and histograms == cache.get('histograms') and cache.get('simple_output'):
    output = cache.get('simple_output')
    
else:
    print ("I'm running now")

    output = processor.run_uproot_job(
        fileset,
        "Events",
        dielectron_mass(year=year, variations=[], accumulator=desired_output),
        exe,
        exe_args,
        chunksize=250000,
    )

    cache['fileset']        = fileset
    cache['cfg']            = cfg
    cache['histograms']     = histograms
    cache['simple_output']  = output
    cache.dump()

In [ ]:
output['totalEvents']['all']/1e6

In [ ]:
import cloudpickle
import gzip
outname = 'dielectron_mc_NLO_'+str(year)
os.system("mkdir -p histos/")
print('Saving output in %s...'%("histos/" + outname + ".pkl.gz"))
with gzip.open("histos/" + outname + ".pkl.gz", "wb") as fout:
    cloudpickle.dump(output, fout)
print('Done!')

In [ ]:
# import the plotting libararies: matplotlib and mplhep

import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

import numpy as np


# load the functions to make a nice plot from the output histograms
# and the scale_and_merge function that scales the individual histograms
# to match the physical cross section

from plots.helpers import makePlot, scale_and_merge

# define a few axes that we can use to rebin our output histograms

N_bins         = hist.Bin('multiplicity', r'$N$', 10, -0.5, 9.5)
N_bins_red     = hist.Bin('multiplicity', r'$N$', 5, -0.5, 4.5)
pt_bins        = hist.Bin('pt', r'$p_{T}\ (GeV)$', np.array([15, 40, 60, 80, 100, 200, 300]))
pt_fine_bins   = hist.Bin('pt', r'$p_{T}\ (GeV)$', 300, 0, 300)
pt_rebin       = hist.Bin('pt', r'$p_{T}\ (GeV)$', 75, 0, 150)
eta_bins       = hist.Bin('eta', r'$\eta $', np.array([0, 0.8, 1.479, 2.5]))
eta_rebin      = hist.Bin('eta', r'$\eta $', 25, -2.5, 2.5)    
phi_bins       = hist.Bin('phi', r'$\phi $', 16, -3.2, 3.2)
mass_bins      = hist.Bin('mass', r'$mass (GeV/c^2)$', 19, 48, 124)

# 1D Histograms

In [ ]:
from yahist import Hist1D, Hist2D

In [ ]:
def get_total(histos, keys):
        tmp = Hist1D.from_bincounts(np.zeros(len(histos[keys[0]].counts)), histos[keys[0]].edges, )
        for key in keys:
            tmp += histos[key]
        return tmp

def add_uncertainty(hist, ax, ratio=False):
    opts = {'step': 'post', 'label': 'Uncertainty', 'hatch': '///',
                    'facecolor': 'none', 'edgecolor': (0, 0, 0, .5), 'linewidth': 0, 'zorder':10.}
    
    if ratio:
        down = np.ones(len(hist.counts)) - hist.errors/hist.counts
        up = np.ones(len(hist.counts)) + hist.errors/hist.counts
    else:
        down = hist.counts-hist.errors
        up = hist.counts+hist.errors
    ax.fill_between(x=hist.edges, y1=np.r_[down, down[-1]], y2=np.r_[up, up[-1]], **opts)

In [ ]:
tmp1.sum('pt').values()

In [ ]:
tmp1 = output['dilep_mass1'].copy()
tmp1 = tmp1.rebin('mass', mass_bins)
tmp1 = tmp1.rebin('pt', pt_rebin)

if year == 2016:

    h1_DY = Hist1D.from_bincounts(
        #tmp1.sum('pt').values()[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)].T,
        tmp1.sum('pt').values()[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)].T,
        (tmp1.axis('mass').edges()),
        #errors = np.sqrt(tmp1.sum('pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)][1].T),
        errors = np.sqrt(tmp1.sum('pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)][1].T),
    )

    #h1_DY = h1_DY*6021.0*1000/93007332.0*36 using NLO sumWeight
    h1_DY = h1_DY*6021.0*1000/1897141621849.1206*36
    
    #keep the TT/TW/WT separated here

    h1_TT = Hist1D.from_bincounts(
        tmp1.sum('pt').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp1.axis('mass').edges()),
        errors = np.sqrt(tmp1.sum('pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h1_TT = h1_TT*87.315*1000/4864515502.84892*36
    
    h1_TW = Hist1D.from_bincounts(
        tmp1.sum('pt').values()[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp1.axis('mass').edges()),
        errors = np.sqrt(tmp1.sum('pt').values(sumw2=True)[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )
    
    h1_TW = h1_TW*19.47*1000/35574325.20617753*36
    
    h1_WT = Hist1D.from_bincounts(
        tmp1.sum('pt').values()[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp1.axis('mass').edges()),
        errors = np.sqrt(tmp1.sum('pt').values(sumw2=True)[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )
    
    h1_WT = h1_WT*19.47*1000/357972341.4959772*36
    
    h1_TT_TW = h1_TT + h1_TW + h1_WT
    
    #add them together after scaling
    
    h1_WZ = Hist1D.from_bincounts(
        tmp1.sum('pt').values()[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp1.axis('mass').edges()),
        errors = np.sqrt(tmp1.sum('pt').values(sumw2=True)[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T,)
        )
    
    h1_WZ = h1_WZ*4.666*1000/97448995.340155*36

    h2_DY = Hist1D.from_bincounts(
        #tmp1.sum('mass').values()[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)].T,
        tmp1.sum('mass').values()[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)].T,
        (tmp1.axis('pt').edges()),
        #errors = np.sqrt(tmp1.sum('mass').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)][1].T),
        errors = np.sqrt(tmp1.sum('mass').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)][1].T),
)

    #h2_DY = h2_DY*6021.0*1000/48968256.0*36
    h2_DY = h2_DY*6021.0*1000/1897141621849.1206*36
    
    #keep the TT/TW/WT separated here

    h2_TT = Hist1D.from_bincounts(
        tmp1.sum('mass').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp1.axis('pt').edges()),
        errors = np.sqrt(tmp1.sum('mass',).values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )
    
    h2_TT = h2_TT*87.315*1000/4960123622.722055*36
    
    h2_TW = Hist1D.from_bincounts(
        tmp1.sum('mass').values()[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp1.axis('pt').edges()),
        errors = np.sqrt(tmp1.sum('mass').values(sumw2=True)[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )
    
    h2_TW = h2_TW*19.47*1000/35574325.20617753*36
    
    h2_WT = Hist1D.from_bincounts(
        tmp1.sum('mass').values()[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp1.axis('pt').edges()),
        errors = np.sqrt(tmp1.sum('mass').values(sumw2=True)[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )
    
    h2_WT = h2_WT*19.47*1000/357972341.4959772*36
    
    h2_TT_TW = h2_TT + h2_TW + h2_WT
    
    #add them together after scaling
    
    h2_WZ = Hist1D.from_bincounts(
        tmp1.sum('mass').values()[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp1.axis('pt').edges()),
        errors = np.sqrt(tmp1.sum('mass').values(sumw2=True)[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T,)
        )
    
    h2_WZ = h2_WZ*4.666*1000/97448995.340155*36

if year == 2017:

    h1_DY = Hist1D.from_bincounts(
        tmp1.sum('pt').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)].T,
        (tmp1.axis('mass').edges()),
        errors = np.sqrt(tmp1.sum('pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)][1].T),
    )

    h1_DY = h1_DY*6021.0*1000/48968256.0*60

    h1_TT = Hist1D.from_bincounts(
        tmp1.sum('pt').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
        (tmp1.axis('mass').edges()),
        errors = np.sqrt(tmp1.sum('pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)][1].T),
    )

    h1_TT = h1_TT*87.315*1000/4960123622.722055*60

    h2_DY = Hist1D.from_bincounts(
        tmp1.sum('mass').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)].T,
        (tmp1.axis('pt').edges()),
        errors = np.sqrt(tmp1.sum('mass').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)][1].T),
    )

    h2_DY = h2_DY*6021.0*1000/48968256.0*60

    h2_TT = Hist1D.from_bincounts(
        tmp1.sum('mass').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
        (tmp1.axis('pt').edges()),
        errors = np.sqrt(tmp1.sum('mass',).values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)][1].T),
    )
    
    h2_TT = h2_TT*87.315*1000/4960123622.722055*60

    
if year == 2018:
    h1_DY = Hist1D.from_bincounts(
        tmp1.sum('pt').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp1.axis('mass').edges()),
        errors = np.sqrt(tmp1.sum('pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h1_DY = h1_DY*6021.0*1000/100108281.94794683*60

    h1_TT = Hist1D.from_bincounts(
        tmp1.sum('pt').values()[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp1.axis('mass').edges()),
        errors = np.sqrt(tmp1.sum('pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h1_TT = h1_TT*87.315*1000/4635769336.539396*60
    
    h1_WZ = Hist1D.from_bincounts(
        tmp1.sum('pt').values()[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp1.axis('mass').edges()),
        errors = np.sqrt(tmp1.sum('pt').values(sumw2=True)[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )
    
    h1_WZ = h1_WZ*4.666*1000/81596490.87739207*60

    h2_DY = Hist1D.from_bincounts(
        tmp1.sum('mass').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp1.axis('pt').edges()),
        errors = np.sqrt(tmp1.sum('mass').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h2_DY = h2_DY*6021.0*1000/100108281.94794683*60

    h2_TT = Hist1D.from_bincounts(
        tmp1.sum('mass').values()[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp1.axis('pt').edges()),
        errors = np.sqrt(tmp1.sum('mass',).values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )
    
    h2_TT = h2_TT*87.315*1000/4635769336.539396*60
    
    h2_WZ = Hist1D.from_bincounts(
        tmp1.sum('mass').values()[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp1.axis('pt').edges()),
        errors = np.sqrt(tmp1.sum('mass').values(sumw2=True)[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )
    
    h2_WZ = h2_WZ*4.666*1000/81596490.87739207*60

In [ ]:
hists = {'mass_DY_OS': h1_DY,
         'mass_TT_TW_OS': h1_TT_TW,
         'mass_WZ_OS': h1_WZ,
}

hists['mass_TT_TW_OS'].label = '$TT/TW\ (OS)$'
hists['mass_TT_TW_OS'].color = '#8AC926'

hists['mass_DY_OS'].label = '$DY\ (OS)$'
hists['mass_DY_OS'].color = '#FFCA3A'

hists['mass_WZ_OS'].label = '$WZ\ (OS)$'
hists['mass_WZ_OS'].color = '#C93126'

keys = ['mass_WZ_OS', 'mass_TT_TW_OS', 'mass_DY_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['mass_OS_data']).divide(hists['mass_OS_TT']+hists['mass_OS_DY'])

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=36.0,
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['mass_DY_OS'].edges,
    w2=[(hists[x].errors)**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

#hep.histplot(
#    [hists['mass_OS_data'].counts],
#    hists['mass_OS_data'].edges,
#    w2=(hists['mass_OS_data'].errors)**2,
#    histtype="errorbar",
#    stack=False,
#    label=['%s (%.0f)'%('Data (OS)', np.sum(hists['mass_OS_data'].counts))],
#    color=['#525B76'],
#    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$mass_{ee}\ (GeV)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

add_uncertainty(total_mc, ax, ratio=True)

ax.legend(loc=2)

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
hists = {'pt_DY_OS': h2_DY,
         'pt_TT_TW_OS': h2_TT_TW,
         'pt_WZ_OS': h2_WZ,
}

hists['pt_TT_TW_OS'].label = '$TT/TW\ (OS)$'
hists['pt_TT_TW_OS'].color = '#8AC926'

hists['pt_DY_OS'].label = '$DY\ (OS)$'
hists['pt_DY_OS'].color = '#FFCA3A'

hists['pt_WZ_OS'].label = '$WZ\ (OS)$'
hists['pt_WZ_OS'].color = '#C93126'

keys = ['pt_WZ_OS', 'pt_TT_TW_OS', 'pt_DY_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['mass_OS_data']).divide(hists['mass_OS_TT']+hists['mass_OS_DY'])

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=60.0,
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['pt_DY_OS'].edges,
    w2=[(hists[x].errors)**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

#hep.histplot(
#    [hists['mass_OS_data'].counts],
#    hists['mass_OS_data'].edges,
#    w2=(hists['mass_OS_data'].errors)**2,
#    histtype="errorbar",
#    stack=False,
#    label=['%s (%.0f)'%('Data (OS)', np.sum(hists['mass_OS_data'].counts))],
#    color=['#525B76'],
#    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$p_{T_{ee}}\ (GeV)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

add_uncertainty(total_mc, ax, ratio=True)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
tmp3 = output['electron_data1'].copy()
tmp3 = tmp3.rebin('pt', pt_rebin)
tmp3 = tmp3.rebin('eta', eta_rebin)


tmp4 = output['electron_data2'].copy()
tmp4 = tmp4.rebin('pt', pt_rebin)
tmp4 = tmp4.rebin('eta', eta_rebin)

if year == 2016:
    h1_DY = Hist1D.from_bincounts(
        tmp3.sum('phi', 'eta').values()[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)].T,
        (tmp3.axis('pt').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'eta').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)][1].T),
    )

    h2_DY = Hist1D.from_bincounts(
        tmp4.sum('phi', 'eta').values()[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)].T,
        (tmp4.axis('pt').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'eta').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)][1].T),
    )

    h3_DY = Hist1D.from_bincounts(
        tmp3.sum('phi', 'pt').values()[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)].T,
        (tmp3.axis('eta').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)][1].T),
    )

    h4_DY = Hist1D.from_bincounts(
        tmp4.sum('phi', 'pt').values()[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)].T,
        (tmp4.axis('eta').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)][1].T),
    )

    h5_DY = Hist1D.from_bincounts(
        tmp3.sum('eta', 'pt').values()[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)].T,
        (tmp3.axis('phi').edges()),
        errors = np.sqrt(tmp3.sum('eta', 'pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)][1].T),
    )

    h6_DY = Hist1D.from_bincounts(
        tmp4.sum('eta', 'pt').values()[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)].T,
        (tmp4.axis('phi').edges()),
        errors = np.sqrt(tmp4.sum('eta', 'pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8_ext2-v1/NANOAODSIM',)][1].T),
    )

    #h1_DY = h1_DY*6021.0*1000/93007332.0*36
    #h2_DY = h2_DY*6021.0*1000/93007332.0*36
    #h3_DY = h3_DY*6021.0*1000/93007332.0*36
    #h4_DY = h4_DY*6021.0*1000/93007332.0*36
    #h5_DY = h5_DY*6021.0*1000/93007332.0*36
    #h6_DY = h6_DY*6021.0*1000/93007332.0*36
    h1_DY = h1_DY*6021.0*1000/1897141621849.1206*36
    h2_DY = h2_DY*6021.0*1000/1897141621849.1206*36
    h3_DY = h3_DY*6021.0*1000/1897141621849.1206*36
    h4_DY = h4_DY*6021.0*1000/1897141621849.1206*36
    h5_DY = h5_DY*6021.0*1000/1897141621849.1206*36
    h6_DY = h6_DY*6021.0*1000/1897141621849.1206*36


    h1_TT = Hist1D.from_bincounts(
        tmp3.sum('phi', 'eta').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('pt').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'eta').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h2_TT = Hist1D.from_bincounts(
        tmp4.sum('phi', 'eta').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('pt').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'eta').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h3_TT = Hist1D.from_bincounts(
        tmp3.sum('phi', 'pt').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('eta').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h4_TT = Hist1D.from_bincounts(
        tmp4.sum('phi', 'pt').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('eta').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h5_TT = Hist1D.from_bincounts(
        tmp3.sum('eta', 'pt').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('phi').edges()),
        errors = np.sqrt(tmp3.sum('eta', 'pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h6_TT = Hist1D.from_bincounts(
        tmp4.sum('eta', 'pt').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('phi').edges()),
        errors = np.sqrt(tmp4.sum('eta', 'pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h1_TT = h1_TT*87.315*1000/4864515502.84892*36
    h2_TT = h2_TT*87.315*1000/4864515502.84892*36
    h3_TT = h3_TT*87.315*1000/4864515502.84892*36
    h4_TT = h4_TT*87.315*1000/4864515502.84892*36
    h5_TT = h5_TT*87.315*1000/4864515502.84892*36
    h6_TT = h6_TT*87.315*1000/4864515502.84892*36
    
    
    h1_TW = Hist1D.from_bincounts(
        tmp3.sum('phi', 'eta').values()[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('pt').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'eta').values(sumw2=True)[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h2_TW = Hist1D.from_bincounts(
        tmp4.sum('phi', 'eta').values()[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('pt').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'eta').values(sumw2=True)[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h3_TW = Hist1D.from_bincounts(
        tmp3.sum('phi', 'pt').values()[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('eta').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'pt').values(sumw2=True)[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h4_TW = Hist1D.from_bincounts(
        tmp4.sum('phi', 'pt').values()[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('eta').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'pt').values(sumw2=True)[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h5_TW = Hist1D.from_bincounts(
        tmp3.sum('eta', 'pt').values()[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('phi').edges()),
        errors = np.sqrt(tmp3.sum('eta', 'pt').values(sumw2=True)[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h6_TW = Hist1D.from_bincounts(
        tmp4.sum('eta', 'pt').values()[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('phi').edges()),
        errors = np.sqrt(tmp4.sum('eta', 'pt').values(sumw2=True)[('/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h1_TW = h1_TW*19.47*1000/35574325.20617753*36
    h2_TW = h2_TW*19.47*1000/35574325.20617753*36
    h3_TW = h3_TW*19.47*1000/35574325.20617753*36
    h4_TW = h4_TW*19.47*1000/35574325.20617753*36
    h5_TW = h5_TW*19.47*1000/35574325.20617753*36
    h6_TW = h6_TW*19.47*1000/35574325.20617753*36

    
    h1_WT = Hist1D.from_bincounts(
        tmp3.sum('phi', 'eta').values()[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('pt').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'eta').values(sumw2=True)[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h2_WT = Hist1D.from_bincounts(
        tmp4.sum('phi', 'eta').values()[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('pt').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'eta').values(sumw2=True)[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h3_WT = Hist1D.from_bincounts(
        tmp3.sum('phi', 'pt').values()[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('eta').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'pt').values(sumw2=True)[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h4_WT = Hist1D.from_bincounts(
        tmp4.sum('phi', 'pt').values()[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('eta').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'pt').values(sumw2=True)[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h5_WT = Hist1D.from_bincounts(
        tmp3.sum('eta', 'pt').values()[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('phi').edges()),
        errors = np.sqrt(tmp3.sum('eta', 'pt').values(sumw2=True)[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h6_WT = Hist1D.from_bincounts(
        tmp4.sum('eta', 'pt').values()[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('phi').edges()),
        errors = np.sqrt(tmp4.sum('eta', 'pt').values(sumw2=True)[('/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIISummer16NanoAODv7-Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h1_WT = h1_WT*19.47*1000/357972341.4959772*36
    h2_WT = h2_WT*19.47*1000/357972341.4959772*36
    h3_WT = h3_WT*19.47*1000/357972341.4959772*36
    h4_WT = h4_WT*19.47*1000/357972341.4959772*36
    h5_WT = h5_WT*19.47*1000/357972341.4959772*36
    h6_WT = h6_WT*19.47*1000/357972341.4959772*36
    
    
    h1_TT_TW = h1_TT + h1_TW + h1_WT
    h2_TT_TW = h2_TT + h2_TW + h2_WT
    h3_TT_TW = h3_TT + h3_TW + h3_WT
    h4_TT_TW = h4_TT + h4_TW + h4_WT
    h5_TT_TW = h5_TT + h5_TW + h5_WT
    h6_TT_TW = h6_TT + h6_TW + h6_WT
    
    
    h1_WZ = Hist1D.from_bincounts(
        tmp3.sum('phi', 'eta').values()[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('pt').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'eta').values(sumw2=True)[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h2_WZ = Hist1D.from_bincounts(
        tmp4.sum('phi', 'eta').values()[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('pt').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'eta').values(sumw2=True)[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h3_WZ = Hist1D.from_bincounts(
        tmp3.sum('phi', 'pt').values()[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('eta').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'pt').values(sumw2=True)[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )


    h4_WZ = Hist1D.from_bincounts(
        tmp4.sum('phi', 'pt').values()[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('eta').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'pt').values(sumw2=True)[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )

    h5_WZ = Hist1D.from_bincounts(
        tmp3.sum('eta', 'pt').values()[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('phi').edges()),
        errors = np.sqrt(tmp3.sum('eta', 'pt').values(sumw2=True)[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )


    h6_WZ = Hist1D.from_bincounts(
        tmp4.sum('eta', 'pt').values()[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('phi').edges()),
        errors = np.sqrt(tmp4.sum('eta', 'pt').values(sumw2=True)[('/WZTo3LNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM',)][1].T),
    )
    
    h1_WZ = h1_WZ*4.666*1000/97448995.340155*36
    h2_WZ = h2_WZ*4.666*1000/97448995.340155*60
    h3_WZ = h3_WZ*4.666*1000/97448995.340155*60
    h4_WZ = h4_WZ*4.666*1000/97448995.340155*60
    h5_WZ = h5_WZ*4.666*1000/97448995.340155*60
    h6_WZ = h6_WZ*4.666*1000/97448995.340155*60

if year == 2017:
    h1_DY = Hist1D.from_bincounts(
        tmp3.sum('phi', 'eta').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)].T,
        (tmp3.axis('pt').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'eta').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)][1].T),
    )

    h2_DY = Hist1D.from_bincounts(
        tmp4.sum('phi', 'eta').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)].T,
        (tmp4.axis('pt').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'eta').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)][1].T),
    )

    h3_DY = Hist1D.from_bincounts(
        tmp3.sum('phi', 'pt').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)].T,
        (tmp3.axis('eta').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)][1].T),
    )

    h4_DY = Hist1D.from_bincounts(
        tmp4.sum('phi', 'pt').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)].T,
        (tmp4.axis('eta').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)][1].T),
    )

    h5_DY = Hist1D.from_bincounts(
        tmp3.sum('eta', 'pt').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)].T,
        (tmp3.axis('phi').edges()),
        errors = np.sqrt(tmp3.sum('eta', 'pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)][1].T),
    )

    h6_DY = Hist1D.from_bincounts(
        tmp4.sum('eta', 'pt').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)].T,
        (tmp4.axis('phi').edges()),
        errors = np.sqrt(tmp4.sum('eta', 'pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)][1].T),
    )

    h1_DY = h1_DY*6021.0*1000/48968256.0*60
    h2_DY = h2_DY*6021.0*1000/48968256.0*60
    h3_DY = h3_DY*6021.0*1000/48968256.0*60
    h4_DY = h4_DY*6021.0*1000/48968256.0*60
    h5_DY = h5_DY*6021.0*1000/48968256.0*60
    h6_DY = h6_DY*6021.0*1000/48968256.0*60


    h1_TT = Hist1D.from_bincounts(
        tmp3.sum('phi', 'eta').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('pt').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'eta').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)][1].T),
    )

    h2_TT = Hist1D.from_bincounts(
        tmp4.sum('phi', 'eta').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('pt').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'eta').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)][1].T),
    )

    h3_TT = Hist1D.from_bincounts(
        tmp3.sum('phi', 'pt').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('eta').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)][1].T),
    )

    h4_TT = Hist1D.from_bincounts(
        tmp4.sum('phi', 'pt').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('eta').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)][1].T),
    )

    h5_TT = Hist1D.from_bincounts(
        tmp3.sum('eta', 'pt').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
        (tmp3.axis('phi').edges()),
        errors = np.sqrt(tmp3.sum('eta', 'pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)][1].T),
    )

    h6_TT = Hist1D.from_bincounts(
        tmp4.sum('eta', 'pt').values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
        (tmp4.axis('phi').edges()),
        errors = np.sqrt(tmp4.sum('eta', 'pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)][1].T),
    )

    h1_TT = h1_TT*87.315*1000/4960123622.722055*60
    h2_TT = h2_TT*87.315*1000/4960123622.722055*60
    h3_TT = h3_TT*87.315*1000/4960123622.722055*60
    h4_TT = h4_TT*87.315*1000/4960123622.722055*60
    h5_TT = h5_TT*87.315*1000/4960123622.722055*60
    h6_TT = h6_TT*87.315*1000/4960123622.722055*60
    
if year == 2018:
    h1_DY = Hist1D.from_bincounts(
        tmp3.sum('phi', 'eta').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp3.axis('pt').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'eta').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h2_DY = Hist1D.from_bincounts(
        tmp4.sum('phi', 'eta').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp4.axis('pt').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'eta').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h3_DY = Hist1D.from_bincounts(
        tmp3.sum('phi', 'pt').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp3.axis('eta').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h4_DY = Hist1D.from_bincounts(
        tmp4.sum('phi', 'pt').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp4.axis('eta').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h5_DY = Hist1D.from_bincounts(
        tmp3.sum('eta', 'pt').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp3.axis('phi').edges()),
        errors = np.sqrt(tmp3.sum('eta', 'pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h6_DY = Hist1D.from_bincounts(
        tmp4.sum('eta', 'pt').values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp4.axis('phi').edges()),
        errors = np.sqrt(tmp4.sum('eta', 'pt').values(sumw2=True)[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h1_DY = h1_DY*6021.0*1000/100108281.94794683*60
    h2_DY = h2_DY*6021.0*1000/100108281.94794683*60
    h3_DY = h3_DY*6021.0*1000/100108281.94794683*60
    h4_DY = h4_DY*6021.0*1000/100108281.94794683*60
    h5_DY = h5_DY*6021.0*1000/100108281.94794683*60
    h6_DY = h6_DY*6021.0*1000/100108281.94794683*60


    h1_TT = Hist1D.from_bincounts(
        tmp3.sum('phi', 'eta').values()[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp3.axis('pt').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'eta').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h2_TT = Hist1D.from_bincounts(
        tmp4.sum('phi', 'eta').values()[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp4.axis('pt').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'eta').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h3_TT = Hist1D.from_bincounts(
        tmp3.sum('phi', 'pt').values()[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp3.axis('eta').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h4_TT = Hist1D.from_bincounts(
        tmp4.sum('phi', 'pt').values()[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp4.axis('eta').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h5_TT = Hist1D.from_bincounts(
        tmp3.sum('eta', 'pt').values()[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp3.axis('phi').edges()),
        errors = np.sqrt(tmp3.sum('eta', 'pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h6_TT = Hist1D.from_bincounts(
        tmp4.sum('eta', 'pt').values()[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp4.axis('phi').edges()),
        errors = np.sqrt(tmp4.sum('eta', 'pt').values(sumw2=True)[('/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h1_TT = h1_TT*87.315*1000/4635769336.539396*60
    h2_TT = h2_TT*87.315*1000/4635769336.539396*60
    h3_TT = h3_TT*87.315*1000/4635769336.539396*60
    h4_TT = h4_TT*87.315*1000/4635769336.539396*60
    h5_TT = h5_TT*87.315*1000/4635769336.539396*60
    h6_TT = h6_TT*87.315*1000/4635769336.539396*60
    
    h1_WZ = Hist1D.from_bincounts(
        tmp3.sum('phi', 'eta').values()[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp3.axis('pt').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'eta').values(sumw2=True)[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h2_WZ = Hist1D.from_bincounts(
        tmp4.sum('phi', 'eta').values()[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp4.axis('pt').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'eta').values(sumw2=True)[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h3_WZ = Hist1D.from_bincounts(
        tmp3.sum('phi', 'pt').values()[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp3.axis('eta').edges()),
        errors = np.sqrt(tmp3.sum('phi', 'pt').values(sumw2=True)[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )


    h4_WZ = Hist1D.from_bincounts(
        tmp4.sum('phi', 'pt').values()[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp4.axis('eta').edges()),
        errors = np.sqrt(tmp4.sum('phi', 'pt').values(sumw2=True)[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )

    h5_WZ = Hist1D.from_bincounts(
        tmp3.sum('eta', 'pt').values()[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp3.axis('phi').edges()),
        errors = np.sqrt(tmp3.sum('eta', 'pt').values(sumw2=True)[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )


    h6_WZ = Hist1D.from_bincounts(
        tmp4.sum('eta', 'pt').values()[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)].T,
        (tmp4.axis('phi').edges()),
        errors = np.sqrt(tmp4.sum('eta', 'pt').values(sumw2=True)[('/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM',)][1].T),
    )
    
    h1_WZ = h1_WZ*4.666*1000/81596490.87739207*60
    h2_WZ = h2_WZ*4.666*1000/81596490.87739207*60
    h3_WZ = h3_WZ*4.666*1000/81596490.87739207*60
    h4_WZ = h4_WZ*4.666*1000/81596490.87739207*60
    h5_WZ = h5_WZ*4.666*1000/81596490.87739207*60
    h6_WZ = h6_WZ*4.666*1000/81596490.87739207*60

In [ ]:
hists = {
    'pt_DY1_OS': h1_DY,
    'pt_TT_TW1_OS': h1_TT_TW,
    'pt_WZ1_OS': h1_WZ,
}

hists['pt_DY1_OS'].label = '$DY\ (OS)$'
hists['pt_DY1_OS'].color = '#FFCA3A'

hists['pt_TT_TW1_OS'].label = '$TT/TW\ (OS)$'
hists['pt_TT_TW1_OS'].color = '#8AC926'

hists['pt_WZ1_OS'].label = '$WZ\ (OS)$'
hists['pt_WZ1_OS'].color = '#C93126'

keys = ['pt_WZ1_OS', 'pt_TT_TW1_OS', 'pt_DY1_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['pt_OS_data1']).divide(hists['pt_OS_TT1']+hists['pt_OS_DY1'])

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=36,
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['pt_DY1_OS'].edges,
    w2=[(hists[x].errors)**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

#hep.histplot(
#    [hists['mass_OS_data'].counts],
#    hists['mass_OS_data'].edges,
#    w2=(hists['mass_OS_data'].errors)**2,
#    histtype="errorbar",
#    stack=False,
#    label=['%s (%.0f)'%('Data (OS)', np.sum(hists['mass_OS_data'].counts))],
#    color=['#525B76'],
#    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$p_{T}\ (leading\ electron)\ (GeV)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

add_uncertainty(total_mc, ax, ratio=True)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
hists = {
    'pt_DY2_OS': h2_DY,
    'pt_TT_TW2_OS': h2_TT_TW,
    'pt_WZ2_OS': h2_WZ,
}

hists['pt_DY2_OS'].label = '$DY\ (OS)$'
hists['pt_DY2_OS'].color = '#FFCA3A'

hists['pt_TT_TW2_OS'].label = '$TT/TW\ (OS)$'
hists['pt_TT_TW2_OS'].color = '#8AC926'

hists['pt_WZ2_OS'].label = '$WZ\ (OS)$'
hists['pt_WZ2_OS'].color = '#C93126'

keys = ['pt_WZ2_OS', 'pt_TT_TW2_OS', 'pt_DY2_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['pt_OS_data1']).divide(hists['pt_OS_TT1']+hists['pt_OS_DY1'])

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=36,
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['pt_DY2_OS'].edges,
    w2=[(hists[x].errors)**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

#hep.histplot(
#    [hists['mass_OS_data'].counts],
#    hists['mass_OS_data'].edges,
#    w2=(hists['mass_OS_data'].errors)**2,
#    histtype="errorbar",
#    stack=False,
#    label=['%s (%.0f)'%('Data (OS)', np.sum(hists['mass_OS_data'].counts))],
#    color=['#525B76'],
#    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$p_{T}\ (trailing\ electron)\ (GeV)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

add_uncertainty(total_mc, ax, ratio=True)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
hists = {
    'eta_DY1_OS': h3_DY,
    'eta_TT_TW1_OS': h3_TT_TW,
    'eta_WZ1_OS': h3_WZ,
}

hists['eta_DY1_OS'].label = '$DY\ (OS)$'
hists['eta_DY1_OS'].color = '#FFCA3A'

hists['eta_TT_TW1_OS'].label = '$TT/TW\ (OS)$'
hists['eta_TT_TW1_OS'].color = '#8AC926'

hists['eta_WZ1_OS'].label = '$WZ\ (OS)$'
hists['eta_WZ1_OS'].color = '#C93126'

keys = ['eta_WZ1_OS', 'eta_TT_TW1_OS', 'eta_DY1_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['pt_OS_data1']).divide(hists['pt_OS_TT1']+hists['pt_OS_DY1'])

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=36,
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['eta_DY1_OS'].edges,
    w2=[(hists[x].errors)**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

#hep.histplot(
#    [hists['mass_OS_data'].counts],
#    hists['mass_OS_data'].edges,
#    w2=(hists['mass_OS_data'].errors)**2,
#    histtype="errorbar",
#    stack=False,
#    label=['%s (%.0f)'%('Data (OS)', np.sum(hists['mass_OS_data'].counts))],
#    color=['#525B76'],
#    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$\eta\ (leading\ electron)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

add_uncertainty(total_mc, ax, ratio=True)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
hists = {
    'eta_DY2_OS': h4_DY,
    'eta_TT_TW2_OS': h4_TT_TW,
    'eta_WZ2_OS': h4_WZ,
}

hists['eta_DY2_OS'].label = '$DY\ (OS)$'
hists['eta_DY2_OS'].color = '#FFCA3A'

hists['eta_TT_TW2_OS'].label = '$TT/TW\ (OS)$'
hists['eta_TT_TW2_OS'].color = '#8AC926'

hists['eta_WZ2_OS'].label = '$WZ\ (OS)$'
hists['eta_WZ2_OS'].color = '#C93126'

keys = ['eta_WZ2_OS', 'eta_TT_TW2_OS', 'eta_DY2_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['pt_OS_data1']).divide(hists['pt_OS_TT1']+hists['pt_OS_DY1'])

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=36,
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['eta_DY2_OS'].edges,
    w2=[(hists[x].errors)**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

#hep.histplot(
#    [hists['mass_OS_data'].counts],
#    hists['mass_OS_data'].edges,
#    w2=(hists['mass_OS_data'].errors)**2,
#    histtype="errorbar",
#    stack=False,
#    label=['%s (%.0f)'%('Data (OS)', np.sum(hists['mass_OS_data'].counts))],
#    color=['#525B76'],
#    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$\eta\ (trailing\ electron)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

add_uncertainty(total_mc, ax, ratio=True)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
hists = {
    'phi_DY1_OS': h5_DY,
    'phi_TT_TW1_OS': h5_TT_TW,
    'phi_WZ1_OS': h5_WZ,
}

hists['phi_DY1_OS'].label = '$DY\ (OS)$'
hists['phi_DY1_OS'].color = '#FFCA3A'

hists['phi_TT_TW1_OS'].label = '$TT/TW\ (OS)$'
hists['phi_TT_TW1_OS'].color = '#8AC926'

hists['phi_WZ1_OS'].label = '$WZ\ (OS)$'
hists['phi_WZ1_OS'].color = '#C93126'

keys = ['phi_WZ1_OS', 'phi_TT_TW1_OS', 'phi_DY1_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['pt_OS_data1']).divide(hists['pt_OS_TT1']+hists['pt_OS_DY1'])

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=36,
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['phi_DY1_OS'].edges,
    w2=[(hists[x].errors)**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

#hep.histplot(
#    [hists['mass_OS_data'].counts],
#    hists['mass_OS_data'].edges,
#    w2=(hists['mass_OS_data'].errors)**2,
#    histtype="errorbar",
#    stack=False,
#    label=['%s (%.0f)'%('Data (OS)', np.sum(hists['mass_OS_data'].counts))],
#    color=['#525B76'],
#    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$\phi\ (leading\ electron)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

add_uncertainty(total_mc, ax, ratio=True)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
hists = {
    'phi_DY2_OS': h6_DY,
    'phi_TT_TW2_OS': h6_TT_TW,
    'phi_WZ2_OS': h6_WZ,
}

hists['phi_DY2_OS'].label = '$DY\ (OS)$'
hists['phi_DY2_OS'].color = '#FFCA3A'

hists['phi_TT_TW2_OS'].label = '$TT/TW\ (OS)$'
hists['phi_TT_TW2_OS'].color = '#8AC926'

hists['phi_WZ2_OS'].label = '$WZ\ (OS)$'
hists['phi_WZ2_OS'].color = '#C93126'

keys = ['phi_WZ2_OS', 'phi_TT_TW2_OS', 'phi_DY2_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['pt_OS_data1']).divide(hists['pt_OS_TT1']+hists['pt_OS_DY1'])

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=36,
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['phi_DY2_OS'].edges,
    w2=[(hists[x].errors)**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

#hep.histplot(
#    [hists['mass_OS_data'].counts],
#    hists['mass_OS_data'].edges,
#    w2=(hists['mass_OS_data'].errors)**2,
#    histtype="errorbar",
#    stack=False,
#    label=['%s (%.0f)'%('Data (OS)', np.sum(hists['mass_OS_data'].counts))],
#    color=['#525B76'],
#    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$\phi\ (trailing\ electron)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

add_uncertainty(total_mc, ax, ratio=True)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')